# Table of content

1. Data processing and manipulation
2. Model training with initial setting
3. RandomizedSearchCv to search for best hyper parameter
4. Model training with tuned setting
5. Model error comparison between initial setting and tuned hyper-parameter model
6. Tuned-model feature importance visualization

### Part 1:Data processing and manipulation

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# import janestreet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import time
# from xgboost import XGBRegressor# Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
chunksize = 10 ** 6
filename = r'C:/Users/User/Desktop/Kaggle Dataset/train.csv'
data_chunk = []
start_time = time.time()
data_chunk = pd.read_csv(filename)
print("--- %s seconds ---" % (time.time() - start_time))

--- 47.65514349937439 seconds ---


***
Start with processing the label , since evaluation is only fixed to either buy or pass

Define a function to return either 0 = Pass , 1 = Buy 

**Buy = weight * resp > 0**

**Pass = weight * resp <= 0** 
***

In [4]:
def buy_or_pass(df):
    if df['action'] > 0:
        return 1
    else:
        return 0

Process on feature and action for model learning

In [5]:
# Data mentioned the return will be based on weight and resp columns . create another column call return 
def feature_action_split(dataframe_market):
    '''
    Input : Sample dataframe from Jane market prediction data
    Output : feature = not response , weight , date or ts_id
             action = 0 for pass and 1 for buy
    '''
#     dataframe_market = dataframe_market[dataframe_market['weight'] > 0]
    dataframe_market['action'] = dataframe_market['weight']*dataframe_market['resp']
    dataframe_market['action'] = dataframe_market.apply(buy_or_pass,axis=1) 
    feature = dataframe_market.drop(['date','weight','resp_1','resp_2','resp_3','resp_4','resp','ts_id','action'],axis=1)
    print("Features columns : ",feature.columns)
    action = dataframe_market[['action']]
    print("Action counts : \n",action.value_counts())
    return feature,action
data_chunk = data_chunk[data_chunk['weight'] > 0]
feature,action = feature_action_split(data_chunk)


Features columns :  Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_120', 'feature_121', 'feature_122', 'feature_123',
       'feature_124', 'feature_125', 'feature_126', 'feature_127',
       'feature_128', 'feature_129'],
      dtype='object', length=130)
Action counts : 
 action
1         999387
0         981900
dtype: int64


In [11]:
conts = np.stack([feature[col].values for col in feature.columns],1)

In [15]:
conts = torch.tensor(conts,dtype=torch.float)

In [19]:
conts.shape

torch.Size([1981287, 130])

In [20]:
y = torch.tensor(action.values.flatten())

In [22]:
y.shape

torch.Size([1981287])

In [8]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
torch.manual_seed(33)
model = TabularModel(conts.shape[1], 2, [260,130], p=0.4) # out_sz = 2

In [57]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [58]:
batch_size = 60000
test_size = 12000

con_train = conts[:batch_size-test_size]
con_test = conts[batch_size-test_size:batch_size]
y_train = y[:batch_size-test_size]
y_test = y[batch_size-test_size:batch_size]

In [55]:
class TabularModel(nn.Module):

    def __init__(self, n_cont, out_sz, layers, p=0.5):
        super().__init__()

        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_in = n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self,  x_cont):

        x = self.bn_cont(x_cont)
        print(x)
        x = self.layers(x)
        return x

In [59]:
con_train.shape

torch.Size([48000, 130])

In [60]:
import time
start_time = time.time()

epochs = 300
losses = []

for i in range(epochs):
    i+=1
    y_pred = model(con_train)
    print(y_pred)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%25 == 1:
        print(f'epoch: {i:3}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'epoch: {i:3}  loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

tensor([[-0.9924, -0.7263, -0.7164,  ...,     nan,     nan,     nan],
        [ 1.0077, -1.3861, -1.3704,  ...,     nan,     nan,     nan],
        [-0.9924, -1.3861, -1.3704,  ...,     nan,     nan,     nan],
        ...,
        [ 1.0077, -0.5251, -0.6452,  ...,     nan,     nan,     nan],
        [ 1.0077,  0.7037,  0.2501,  ...,     nan,     nan,     nan],
        [ 1.0077, -0.4264, -0.5451,  ...,     nan,     nan,     nan]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)
epoch:   1  loss:        nan
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<NativeBatchNormBackward>)
tens

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[nan, nan]

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[nan, nan]

KeyboardInterrupt: 